In [1]:
from datetime import date as dt
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import io

# Loading in data + discovery

In [2]:
raiders_data = pd.read_csv("raiders.csv")
raiders_data = pd.DataFrame(raiders_data)
raiders_data.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Points,Points.1,...,Def Rank.1,Overall Rank,Overall Rank.1,Overall Rank.2,Overall Rank.3,Simple Rating System,Simple Rating System.1,Simple Rating System.2,Simple Rating System.3,Simple Rating System.4
0,Year,Lg,Tm,W,L,T,Div. Finish,Playoffs,PF,PA,...,Yds,T/G,Pts±,Yds±,out of,MoV,SoS,SRS,OSRS,DSRS
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1994,NFL,Los Angeles Raiders,9,7,0,3rd of 5,NaN,303,327,...,10,22,15,14,28,-1.5,-0.1,-1.6,-1.9,0.3
3,1993,NFL,Los Angeles Raiders*,10,6,0,2nd of 5,Lost Div,306,326,...,9,18,18,10,28,-1.3,0.6,-0.7,1.1,-1.7
4,1992,NFL,Los Angeles Raiders,7,9,0,4th of 5,NaN,249,281,...,9,28,15,14,28,-2.0,0.4,-1.6,-3.2,1.5


In [3]:
# Getting first (0th) row of the "Unnamed: 0" col
raiders_data.loc[0, 'Unnamed: 0']

'Year'

# Fixing Column Names


In [4]:
# range function specifies the amount of time to run the loop for col number
for i in range(raiders_data.shape[1]):
    col = raiders_data.columns[i]
    raiders_data.rename(columns = {col: raiders_data.loc[0,col]}, inplace = True)

# getting rid of the first 2 rows
raiders_data.drop([raiders_data.index[0], raiders_data.index[1]], inplace = True)

raiders_data.head(5)

,Year,Lg,Tm,W,L,T,Div. Finish,Playoffs,PF,PA,...,Yds,T/G,Pts±,Yds±,out of,MoV,SoS,SRS,OSRS,DSRS
2,1994,NFL,Los Angeles Raiders,9,7,0,3rd of 5,NaN,303,327,...,10,22,15,14,28,-1.5,-0.1,-1.6,-1.9,0.3
3,1993,NFL,Los Angeles Raiders*,10,6,0,2nd of 5,Lost Div,306,326,...,9,18,18,10,28,-1.3,0.6,-0.7,1.1,-1.7
4,1992,NFL,Los Angeles Raiders,7,9,0,4th of 5,NaN,249,281,...,9,28,15,14,28,-2.0,0.4,-1.6,-3.2,1.5
5,1991,NFL,Los Angeles Raiders*,9,7,0,3rd of 5,Lost WC,298,297,...,21,11,15,24,28,0.1,1.3,1.4,0.3,1.1
6,1990,NFL,Los Angeles Raiders*,12,4,0,1st of 5,Lost Conf,337,268,...,4,14,9,13,28,4.3,2.2,6.6,2.2,4.4


# Creating a column_fixer function

In [5]:
def column_fixer(data):
    # range function specifies the amount of time to run the loop for col number
    for i in range(data.shape[1]):
        col = data.columns[i]
        data.rename(columns = {col: data.loc[0,col]}, inplace = True)

    # getting rid of the first 2 rows
    data.drop([data.index[0], data.index[1]], inplace = True)

# Fixing other NFL datasets

In [6]:
chargers_data = pd.read_csv('chargers.csv')
column_fixer(chargers_data)

rams_data = pd.read_csv('rams.csv')
column_fixer(rams_data)

# Loading in CFB data

In [7]:
ucla_data = pd.read_csv("ucla.csv")
usc_data = pd.read_csv('usc.csv')

In [14]:
ucla_data.shape

(95, 19)

In [9]:
# adding 'ucla' prefixes to all col names
for i in range(ucla_data.shape[1]):
    col = ucla_data.columns[i]
    if col == 'Year':
        continue
    ucla_data.rename(columns = {col: ('ucla '+ col)}, inplace = True)
    
ucla_data.columns

Index(['Rk', 'Year', 'ucla Conf', 'ucla Overall W', 'ucla Overall L',
       'ucla Overall T', 'ucla Overall Pct', 'ucla Conference W',
       'ucla Conference L', 'ucla Conference T', 'ucla Conference Pct',
       'ucla SRS', 'ucla SOS', 'ucla AP Pre', 'ucla AP High', 'ucla AP Post',
       'ucla Coach(es)', 'ucla Bowl', 'ucla Notes'],
      dtype='object')

In [10]:
# adding 'usc' prefixes to all col names
for i in range(usc_data.shape[1]):
    col = usc_data.columns[i]
    if col == 'Year':
        continue
    usc_data.rename(columns = {col: ('usc '+ col)}, inplace = True)
    
usc_data.columns

Index(['Rk', 'Year', 'usc Conf', 'usc Overall W', 'usc Overall L',
       'usc Overall T', 'usc Overall Pct', 'usc Conference W',
       'usc Conference L', 'usc Conference T', 'usc Conference Pct', 'usc SRS',
       'usc SOS', 'usc AP Pre', 'usc AP High', 'usc AP Post', 'usc Coach(es)',
       'usc Bowl', 'usc Notes'],
      dtype='object')

In [12]:
# merging ucla & usc datasets
ucla_and_usc = pd.merge(
    ucla_data, usc_data, on = ['Year']
)

ucla_and_usc.head(5)

,Rk,Year,ucla Conf,ucla Overall W,ucla Overall L,ucla Overall T,ucla Overall Pct,ucla Conference W,ucla Conference L,ucla Conference T,...,usc Conference T,usc Conference Pct,usc SRS,usc SOS,usc AP Pre,usc AP High,usc AP Post,usc Coach(es),usc Bowl,usc Notes
0,1,2022,Pac-12,5,0,NaN,1.000,2,0,0,...,0,1.000,17.37,0.37,14.0,6.0,NaN,Lincoln Riley (5-0),NaN,NaN
1,2,2021,Pac-12,8,4,0.0,0.667,6,3,0,...,0,0.333,-1.70,1.80,15.0,14.0,NaN,Clay Helton (1-1) Donte Williams (3-7),NaN,NaN
2,3,2020,Pac-12,3,4,0.0,0.429,3,4,0,...,0,1.000,5.32,-3.68,17.0,13.0,21.0,Clay Helton (5-1),NaN,NaN
3,4,2019,Pac-12,4,8,0.0,0.333,4,5,0,...,0,0.778,8.87,5.64,NaN,21.0,NaN,Clay Helton (8-5),Holiday Bowl-L,NaN
4,5,2018,Pac-12,3,9,0.0,0.250,3,6,0,...,0,0.444,2.22,3.39,15.0,15.0,NaN,Clay Helton (5-7),NaN,NaN
